In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 87.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 75.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

## Local Inference on GPU 
Model page: https://huggingface.co/bigcode/starcoder2-3b

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/bigcode/starcoder2-3b)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

# Load model and tokenizer

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="bigcode/starcoder2-3b")

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
# model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder2-3b")

In [5]:
import torch
from datasets import load_dataset
import subprocess, tempfile, os, sys
import pandas as pd

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b", trust_remote_code=True)

# Load model safely with automatic device placement and half precision
model = AutoModelForCausalLM.from_pretrained(
    "bigcode/starcoder2-3b",
    trust_remote_code=True,
    device_map="auto",          # automatically splits model between GPU and CPU if needed
    dtype=torch.float16         # use half-precision to save memory
)

# Put model in eval mode
model.eval()


Using device: cuda


Starcoder2ForCausalLM(
  (model): Starcoder2Model(
    (embed_tokens): Embedding(49152, 3072)
    (layers): ModuleList(
      (0-29): 30 x Starcoder2DecoderLayer(
        (self_attn): Starcoder2Attention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=True)
          (k_proj): Linear(in_features=3072, out_features=256, bias=True)
          (v_proj): Linear(in_features=3072, out_features=256, bias=True)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=True)
        )
        (mlp): Starcoder2MLP(
          (c_fc): Linear(in_features=3072, out_features=12288, bias=True)
          (c_proj): Linear(in_features=12288, out_features=3072, bias=True)
          (act): PytorchGELUTanh()
        )
        (input_layernorm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
      )
    )
    (norm): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
    (rot

# **Load Dataset**

In [8]:

dataset = load_dataset("openai/openai_humaneval")
problems = dataset["test"]  # 164 tasks

print("Number of problems:", len(problems))
print("Example problem:\n", problems[0])

README.md: 0.00B [00:00, ?B/s]

openai_humaneval/test-00000-of-00001.par(…):   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

Number of problems: 164
Example problem:
 {'task_id': 'HumanEval/0', 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9

# Code Generation

In [12]:
def generate_code_safe(prompt, temperature=0.2):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, temperature=temperature, max_new_tokens=200)
    code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    del inputs, outputs
    torch.cuda.empty_cache()  # free GPU memory
    return code

# Run Tests

In [13]:
def run_tests(code, test_code):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".py", mode="w") as f:
        f.write(code + "\n\n")
        f.write(test_code + "\n")
        test_file = f.name

    try:
        result = subprocess.run([sys.executable, test_file],
                                capture_output=True, text=True, timeout=5)
        passed = (result.returncode == 0)
    except subprocess.TimeoutExpired:
        passed = False
    finally:
        os.remove(test_file)

    return passed


# Evaluation

In [15]:
def evaluate_subset(n=10, temps=[0.2, 0.8]):
    results = []
    for i in range(n):
        problem = problems[i]
        prompt = problem["prompt"]
        test_code = problem["test"]

        for temp in temps:
            code = generate_code_safe(prompt, temperature=temp)
            passed = run_tests(code, test_code)
            results.append({
                "task_id": problem["task_id"],
                "temp": temp,
                "passed": passed,
                "code_snippet": code[:200]
            })
            print(f"[{problem['task_id']}] temp={temp}, passed={passed}")
    return results
print("Baseline performance - bigcode/starcoder2-3b")
results = evaluate_subset(n=10, temps=[0.2, 0.8])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Baseline performance - bigcode/starcoder2-3b


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
